<a href="https://colab.research.google.com/github/JRamos84/deteccion_fraude/blob/revision/02_Data_Cleaning_and_Preprocessing_selection_feature.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 02 Data Cleaning and Preprocessing
**Objetivo**: Limpiar y preprocesar los datos para que estén listos para el modelado.
**Contenido**:
- Manejo de valores nulos.
- Creación de nuevas características (feature engineering).
- Conversión de tipos de datos.
- Codificación de variables categóricas.
- Normalización y estandarización de las variables.


In [42]:
# Librerías
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
from scipy import stats
import datetime
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

In [43]:
def reduce_mem_usage(df, verbose=True):
    """
    Reduce el uso de memoria de un DataFrame de pandas cambiando los tipos de datos de las columnas numéricas
    a tipos más eficientes sin perder información.

    Parameters:
    df (pd.DataFrame): El DataFrame a optimizar.
    verbose (bool): Si es True, imprime el uso de memoria antes y después de la optimización.

    Returns:
    pd.DataFrame: El DataFrame optimizado.
    """
    # Obtener el uso inicial de memoria
    start_mem = df.memory_usage(deep=True).sum() / 1024**2

    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()

            # Optimización para tipos de datos enteros
            if pd.api.types.is_integer_dtype(df[col]):
                if c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)

            # Optimización para tipos de datos flotantes
            else:
                if c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    # Obtener el uso final de memoria
    end_mem = df.memory_usage(deep=True).sum() / 1024**2
    if verbose:
        print(f'Mem. usage decreased to {end_mem:.2f} Mb ({100 * (start_mem - end_mem) / start_mem:.1f}% reduction)')

    return df

In [44]:
## Importamos las librerias a usar
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:

## Cargamos los dataset de train_transaction y train_identity
df_transaction_train = pd.read_csv(data_dir + '/train_transaction.csv')
df_identity_train = pd.read_csv(data_dir + '/train_identity.csv')
## Reducimos los tamaños de los dataset cambiando los tipos de los datos de las columnas
df_transaction_train = reduce_mem_usage(df_transaction_train)
df_identity_train = reduce_mem_usage(df_identity_train)

Mem. usage decreased to 867.89 Mb (58.7% reduction)
Mem. usage decreased to 138.38 Mb (12.2% reduction)


In [46]:

# Combinar los datasets
dataset = pd.merge(df_transaction_train, df_identity_train, on='TransactionID', how='left')


## Consideración principales del análisis de EDA

### Manejo de valores nulos

- Usaremos las columnas que no superen el 50 de los valores nulos

In [47]:
null_percentages = (dataset.isnull().sum() / len(dataset)) * 100
null_percentages[null_percentages < 50].sort_values(ascending=False)

M4               47.658753
D2               47.549192
V1               47.293494
V10              47.293494
D11              47.293494
                   ...    
C12               0.000000
C13               0.000000
C14               0.000000
isFraud           0.000000
TransactionID     0.000000
Length: 220, dtype: float64

In [48]:
col_util = null_percentages[null_percentages < 50].index.to_list()

In [49]:
dataset = dataset[col_util]
dataset.shape

(590540, 220)

In [50]:
## Eliminamos las filas nulas
dataset.dropna(inplace=True)
dataset.shape

(76416, 220)

- La cantidad de registro dismuyo considerablemente luego de la elimación de las columnas y las filas con datos nulos

### Rango de TransactionAMT y Aplicación del Logaritmo

- Basado en el análisis EDA elinamos los registro con valores en TransactionAMT superior a 5192

In [51]:
dataset = dataset[dataset['TransactionAmt'] < 5192]

In [52]:
dataset['TransactionAmt'] = np.log(dataset['TransactionAmt'])

####Convertiendo 'TransactionDT' en unidades de tiempo

In [53]:



START_DATE = '2017-12-01'
startdate = datetime.datetime.strptime(START_DATE, "%Y-%m-%d")

# Suponiendo que dataset tiene las columnas 'TransactionDT'
dataset["Date"] = dataset['TransactionDT'].apply(lambda x: startdate + datetime.timedelta(seconds=x))

# Crear un DataFrame con las nuevas columnas a agregar
new_columns = pd.DataFrame({
    '_Weekdays': dataset['Date'].dt.dayofweek,
    '_Hours': dataset['Date'].dt.hour,
    '_Days': dataset['Date'].dt.day
})

# Unir todas las columnas al DataFrame original
dataset = pd.concat([dataset, new_columns], axis=1)
dataset['_Weekdays'] =dataset['_Weekdays'].astype('object')
dataset['_Hours'] = dataset['_Hours'].astype('object')
dataset['_Days'] = dataset['_Days'].astype('object')
dataset.drop(['Date','TransactionDT'], axis=1, inplace=True)
dataset.head(3)

,TransactionID,isFraud,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,V315,V316,V317,V318,V319,V320,V321,_Weekdays,_Hours,_Days
9,2987009,0,4.761719,W,17399,111.0,150.0,mastercard,224.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,2
36,2987036,0,4.343750,W,4806,490.0,150.0,visa,226.0,debit,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5,0,2
41,2987041,0,6.652344,W,9002,453.0,150.0,visa,226.0,debit,...,0.0,0.0,0.0,0.0,774.0,774.0,774.0,5,0,2


### Mapping emails

In [54]:
#https://www.kaggle.com/code/kabure/extensive-eda-and-modeling-xgb-hyperopt?scriptVersionId=18427966&cellId=116
emails = {'gmail': 'google', 'att.net': 'att', 'twc.com': 'spectrum',
          'scranton.edu': 'other', 'optonline.net': 'other', 'hotmail.co.uk': 'microsoft',
          'comcast.net': 'other', 'yahoo.com.mx': 'yahoo', 'yahoo.fr': 'yahoo',
          'yahoo.es': 'yahoo', 'charter.net': 'spectrum', 'live.com': 'microsoft',
          'aim.com': 'aol', 'hotmail.de': 'microsoft', 'centurylink.net': 'centurylink',
          'gmail.com': 'google', 'me.com': 'apple', 'earthlink.net': 'other', 'gmx.de': 'other',
          'web.de': 'other', 'cfl.rr.com': 'other', 'hotmail.com': 'microsoft',
          'protonmail.com': 'other', 'hotmail.fr': 'microsoft', 'windstream.net': 'other',
          'outlook.es': 'microsoft', 'yahoo.co.jp': 'yahoo', 'yahoo.de': 'yahoo',
          'servicios-ta.com': 'other', 'netzero.net': 'other', 'suddenlink.net': 'other',
          'roadrunner.com': 'other', 'sc.rr.com': 'other', 'live.fr': 'microsoft',
          'verizon.net': 'yahoo', 'msn.com': 'microsoft', 'q.com': 'centurylink',
          'prodigy.net.mx': 'att', 'frontier.com': 'yahoo', 'anonymous.com': 'other',
          'rocketmail.com': 'yahoo', 'sbcglobal.net': 'att', 'frontiernet.net': 'yahoo',
          'ymail.com': 'yahoo', 'outlook.com': 'microsoft', 'mail.com': 'other',
          'bellsouth.net': 'other', 'embarqmail.com': 'centurylink', 'cableone.net': 'other',
          'hotmail.es': 'microsoft', 'mac.com': 'apple', 'yahoo.co.uk': 'yahoo', 'netzero.com': 'other',
          'yahoo.com': 'yahoo', 'live.com.mx': 'microsoft', 'ptd.net': 'other', 'cox.net': 'other',
          'aol.com': 'aol', 'juno.com': 'other', 'icloud.com': 'apple'}

us_emails = ['gmail', 'net', 'edu']

# https://www.kaggle.com/c/ieee-fraud-detection/discussion/100499#latest-579654
#for c in ['P_emaildomain', 'R_emaildomain']:
for c in ['P_emaildomain']:
    dataset[c + '_bin'] = dataset[c].map(emails)
    dataset[c + '_suffix'] = dataset[c].map(lambda x: str(x).split('.')[-1])
    dataset[c + '_suffix'] = dataset[c + '_suffix'].map(lambda x: x if str(x) not in us_emails else 'us')

## Elimaamos las columnas 'P_emaildomain', 'R_emaildomain'
dataset.drop(['P_emaildomain'], axis=1, inplace=True)
### IMPORTANTE  'R_emaildomain' fue sacada del analisis debido a que tiene un elavado cantidad de valores nulos


## Separamos datos características (X) y variable objetivo (y)

In [67]:
X = dataset.drop('isFraud', axis=1)  # features
y = dataset['isFraud']  # target

In [72]:
num_cols = X.select_dtypes(include='number').columns.tolist()
cat_cols = X.select_dtypes(include=['object']).columns.tolist()

In [74]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# Copiar el DataFrame original X para evitar modificaciones directas
X_encoded = X.copy()
lbl = LabelEncoder()
X_encoded[cat_cols] = X_encoded[cat_cols].apply(lambda x: lbl.fit_transform(x.astype(str)))

# Concatenar X_encoded con el DataFrame original X y eliminar columnas originales
X = pd.concat([X.drop(cat_cols, axis=1), X_encoded[cat_cols]], axis=1)

# Mostrar los primeros registros de X después de la transformación
print(X.head())

    TransactionID  TransactionAmt     card1  card2     card3     card5  \
9        0.000000        0.531785  0.942824  0.022  0.336066  0.918519   
36       0.000046        0.477151  0.218464  0.780  0.336066  0.933333   
41       0.000054        0.778912  0.459822  0.706  0.336066  0.933333   
46       0.000063        0.418177  0.663561  0.442  0.336066  0.933333   
51       0.000071        0.618075  0.392695  0.522  0.336066  0.933333   

       addr1   addr2        C1        C2  ...   M1   M2   M3   M4   M6  \
9   0.231481  0.8875  0.002976  0.003448  ...  1.0  1.0  1.0  0.0  1.0   
36  0.488426  0.8875  0.020833  0.010345  ...  1.0  0.0  0.0  0.5  1.0   
41  0.370370  0.8875  0.002976  0.013793  ...  1.0  1.0  0.0  0.0  0.0   
46  0.780093  0.8875  0.000000  0.000000  ...  1.0  1.0  1.0  0.0  0.0   
51  0.851852  0.8875  0.008929  0.010345  ...  1.0  1.0  1.0  0.0  1.0   

    _Weekdays  _Hours  _Days  P_emaildomain_bin  P_emaildomain_suffix  
9    0.833333     0.0    0.1          

In [73]:
scaler_min_max = MinMaxScaler()
X[num_cols] = scaler_min_max.fit_transform(X[num_cols])



In [76]:
# Guardar X y y en archivos CSV

#dir_data_processed = '../data/processed'
dir_data_processed = '/content/drive/MyDrive/cursos-analisis-datos/data-science/proyecto/propuestas/propuesta1/ieee-fraud-detection/processed'
ruta_archivo_X = os.path.join(dir_data_processed, 'datos_procesados.csv')
X.to_csv(ruta_archivo_X, index=False, encoding='utf-8')

ruta_archivo_y = os.path.join(dir_data_processed, 'target.csv')
y.to_csv(ruta_archivo_y, index=False, encoding='utf-8')

